# Simple script to analyze Jenkins structure and its API.

### Creating instance of Jenkins server to query data

In [2]:
import jenkins
server = jenkins.Jenkins('https://builds.apache.org/')


### Get a list of all jobs from the server with different folder_length

In [50]:
jobs_0 = server.get_all_jobs(folder_depth = 0)
jobs_1 = server.get_all_jobs(folder_depth = 1)
jobs_2 = server.get_all_jobs(folder_depth = 2)
jobs_3 = server.get_all_jobs(folder_depth = 3)

In [3]:
print(f"Number of \"jobs\": \
    \n\tfolder_length = 0: {len(jobs_0)} \
    \n\tfolder_length = 1: {len(jobs_1)} \
    \n\tfolder_length = 2: {len(jobs_2)} \
    \n\tfolder_length = 3: {len(jobs_3)} \
    ")

jobs_2 == jobs_3

Number of "jobs":     
	folder_length = 0: 2164     
	folder_length = 1: 3517     
	folder_length = 2: 4674     
	folder_length = 3: 4674     


True

We get the same number of jobs when folder_length is 2 or 3.
This means the deepest level where an actual job (with build information) may lie is 2.
Moreover, the structure of jobs is the same for level 2 and 3.

### Job types

In [4]:
job_type = set()

for job in jobs_2:
    job_type.add(job['_class'].split('.')[-1]) 
    
print("Job Types:")
for type in job_type:
    print('\t' + type)

Job Types:
	MavenModuleSet
	WorkflowMultiBranchProject
	OrganizationFolder
	WorkflowJob
	FreeStyleProject
	MatrixProject
	ExternalJob
	Folder


In [5]:
set_w_jobs = set()
set_wo_jobs = set()

for i in range(len(jobs_2)):
    if 'jobs' in jobs_2[i]:
        set_w_jobs.add(jobs_2[i]['_class'].split('.')[-1])
    else:
        set_wo_jobs.add(jobs_2[i]['_class'].split('.')[-1]) 
        
print("Job types that contains \"jobs\" field")
print(set_w_jobs,end = "\n\n")

print("Job types that does not contain \"jobs\" field")
print(set_wo_jobs)

Job types that contains "jobs" field
{'WorkflowMultiBranchProject', 'Folder', 'OrganizationFolder'}

Job types that does not contain "jobs" field
{'MavenModuleSet', 'WorkflowJob', 'FreeStyleProject', 'MatrixProject', 'ExternalJob'}


We can see that if a job contains "jobs" field, then it is of class "Folder", "OrganizationFolder" or "WorkflowMultiBranchProject", which contains the actual jobs that have build information.

In [6]:
for i in range(len(jobs_2)):
    if 'jobs' in jobs_2[i]:
        name = jobs_2[i]['name']
        print(name)


2.2.x
apache
Apache Dubbo
Apache OFBiz
Apache Training (incubating) - Tools - Reveal.JS Template
Apache Tuweni
Archiva-IntegrationTests-Gitbox
Archiva-TLP-Gitbox
bookkeeper-jenkins-testing
brooklyn
brooklyn-client
brooklyn-dist
brooklyn-docs
brooklyn-library
brooklyn-server
brooklyn-ui
Camel
Camel JDK11
Camel JDK12
Camel.website
Chainsaw
commons-lang-pipeline
CouchDB
Docker
edgent-pipeline
FlexJS Pipeline
GH-incubator-zipkin
hadoop-multibranch
HBase Nightly
HBase-Find-Flaky-Tests
HBase-Flaky-Tests
HBase-PreCommit-GitHub-PR
incubator-crail
incubator-sdap
incubator-sdap-nexusproto
incubator-sdap-ningesterpy
IoTDB
IoTDB-Pipeline
jclouds-2.1.x
jclouds-pull-request-jobs
jclouds-with-credentials
Log4jCore
Log4jKotlin
Log4jScala
logging-log4net
LoggingParent
maven-box
Mesos
Mesos-Packaging
mynewt-site
Nemo
netbeans-html4j-TLP
netbeans-maven-TLP
netbeans-TLP
OFBiz-Github-MultiBranch
oozie-master-testing
PLC4X
polygene-java
qpid-python
Royale Pipeline
SDAP
Sling
StreamPipes
struts-annotations
s

### Inspecting a job of class "Folder"

In [5]:
server.get_job_info('Apache Dubbo', depth=2, fetch_all_builds=True)['jobs']

[{'_class': 'org.jenkinsci.plugins.workflow.job.WorkflowJob',
  'actions': [{},
   {},
   {},
   {},
   {},
   {},
   {},
   {},
   {'_class': 'hudson.plugins.jobConfigHistory.JobConfigHistoryProjectAction',
    'jobConfigHistory': []},
   {},
   {},
   {},
   {'_class': 'nectar.plugins.rbac.roles.Roles',
    'filterableRoles': [],
    'roles': []},
   {},
   {},
   {'_class': 'org.jenkinsci.plugins.testresultsanalyzer.TestResultsAnalyzerAction',
    'actions': [{}]},
   {},
   {'_class': 'com.cloudbees.plugins.credentials.ViewCredentialsAction',
    'stores': {}}],
  'description': '',
  'displayName': 'apache-dubbo-admin-snapshot-deployment',
  'displayNameOrNull': None,
  'fullDisplayName': 'Apache Dubbo » apache-dubbo-admin-snapshot-deployment',
  'fullName': 'Apache Dubbo/apache-dubbo-admin-snapshot-deployment',
  'name': 'apache-dubbo-admin-snapshot-deployment',
  'url': 'https://builds.apache.org/job/Apache%20Dubbo/job/apache-dubbo-admin-snapshot-deployment/',
  'buildable': Tru

In [12]:
server.get_job_info_regex('Apache Dubbo', folder_depth=0, depth=2)[0]

True

In [61]:
# Apache Dubbo is a job of class "Folder"
print(server.get_job_info('Apache Dubbo', depth=2)['_class'])

'com.cloudbees.hudson.plugins.folder.Folder'

We can only get info about the job contained within the Folder by its full name with parent folder.
<br/>e.g: Apache Dubbo/apache-dubbo-admin-snapshot-deployment

In [33]:
import re
regex = re.compile(f"^.*dubbo.*$", re.IGNORECASE)
server.get_job_info('Apache Dubbo/apache-dubbo-admin-snapshot-deployment', depth=0)

{'_class': 'org.jenkinsci.plugins.workflow.job.WorkflowJob',
 'actions': [{},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {'_class': 'hudson.plugins.jobConfigHistory.JobConfigHistoryProjectAction'},
  {},
  {},
  {},
  {'_class': 'nectar.plugins.rbac.roles.Roles'},
  {},
  {},
  {'_class': 'org.jenkinsci.plugins.testresultsanalyzer.TestResultsAnalyzerAction'},
  {},
  {'_class': 'com.cloudbees.plugins.credentials.ViewCredentialsAction'}],
 'description': '',
 'displayName': 'apache-dubbo-admin-snapshot-deployment',
 'displayNameOrNull': None,
 'fullDisplayName': 'Apache Dubbo » apache-dubbo-admin-snapshot-deployment',
 'fullName': 'Apache Dubbo/apache-dubbo-admin-snapshot-deployment',
 'name': 'apache-dubbo-admin-snapshot-deployment',
 'url': 'https://builds.apache.org/job/Apache%20Dubbo/job/apache-dubbo-admin-snapshot-deployment/',
 'buildable': True,
 'builds': [{'_class': 'org.jenkinsci.plugins.workflow.job.WorkflowRun',
   'number': 17,
   'url': 'https://builds.apache.org/job/Apac

In [65]:
server.get_job_info('Apache Dubbo', depth=2)

{'_class': 'com.cloudbees.hudson.plugins.folder.Folder',
 'actions': [{},
  {},
  {},
  {'_class': 'nectar.plugins.rbac.roles.Roles',
   'filterableRoles': [],
   'roles': []},
  {},
  {},
  {},
  {'_class': 'hudson.plugins.jobConfigHistory.JobConfigHistoryProjectAction',
   'jobConfigHistory': []},
  {},
  {},
  {},
  {'_class': 'com.cloudbees.plugins.credentials.ViewCredentialsAction',
   'stores': {}}],
 'description': '',
 'displayName': 'Apache Dubbo',
 'displayNameOrNull': 'Apache Dubbo',
 'fullDisplayName': 'Apache Dubbo',
 'fullName': 'Apache Dubbo',
 'name': 'Apache Dubbo',
 'url': 'https://builds.apache.org/job/Apache%20Dubbo/',
 'healthReport': [{'description': 'Average health of 3 items',
   'iconClassName': 'icon-health-80plus',
   'iconUrl': 'health-80plus.png',
   'score': 100},
  {'description': 'Jobs with builds: 3 of 3',
   'iconClassName': 'icon-health-80plus',
   'iconUrl': 'health-80plus.png',
   'score': 100},
  {'description': 'Jobs with successful builds: 3 of 3

### Different return contents based on argument "depth"

In [45]:
f1 = server.get_job_info_regex('Aurora', depth = 1)
f2 = server.get_job_info_regex('Aurora', depth = 2)
f3 = server.get_job_info_regex('Aurora', depth = 3)
f4 = server.get_job_info_regex('Aurora', depth = 4)

# print(f1 == f2)
# print(f3 == f2)
# print(f3 == f4)

f2[1]


{'_class': 'hudson.model.FreeStyleProject',
 'actions': [{},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {},
  {'_class': 'hudson.plugins.jobConfigHistory.JobConfigHistoryProjectAction',
   'jobConfigHistory': []},
  {},
  {},
  {},
  {'_class': 'nectar.plugins.rbac.roles.Roles',
   'filterableRoles': [],
   'roles': []},
  {},
  {},
  {},
  {'_class': 'org.jenkinsci.plugins.testresultsanalyzer.TestResultsAnalyzerAction',
   'actions': [{}]},
  {'_class': 'com.cloudbees.plugins.credentials.ViewCredentialsAction',
   'stores': {}}],
 'description': '',
 'displayName': 'AuroraBot',
 'displayNameOrNull': None,
 'fullDisplayName': 'AuroraBot',
 'fullName': 'AuroraBot',
 'name': 'AuroraBot',
 'url': 'https://builds.apache.org/job/AuroraBot/',
 'buildable': False,
 'builds': [{'_class': 'hudson.model.FreeStyleBuild',
   'actions': [{'_class': 'hudson.model.CauseAction',
     'causes': [{'_class': 'hudson.triggers.TimerTrigger$TimerTriggerCause',
       'shortDescription': '

### SCM

In [42]:
scms = set()
for job in server.get_all_jobs():
    
    name = job['name']
    print(name)
    
    job_info = server.get_job_info(name,depth = 0)
    if "scm" in job_info:
        job_info_scm = job_info['scm']
        scm_class = job_info_scm['_class']
        print("\t\t" + scm_class)
        scms.add(scm_class)
        
print(scms)
    

2.2.x
Abdera-trunk
		hudson.scm.SubversionSCM
Accumulo-1.9
		hudson.plugins.git.GitSCM
Accumulo-2.0
		hudson.plugins.git.GitSCM
Accumulo-Master
		hudson.plugins.git.GitSCM
Accumulo-master-IT
		hudson.plugins.git.GitSCM
Accumulo-Maven-Plugin-Master
		hudson.plugins.git.GitSCM
Accumulo-Maven-Plugin-Pull-Requests
		hudson.plugins.git.GitSCM
Accumulo-PR-Builder
		hudson.plugins.git.GitSCM
Accumulo-Pull-Requests
		hudson.plugins.git.GitSCM
ACE-trunk
		hudson.scm.SubversionSCM
ActiveMQ Protocol Buffer
		hudson.scm.SubversionSCM
ActiveMQ-Artemis-Deploy
		hudson.plugins.git.GitSCM
ActiveMQ-Artemis-Master
		hudson.plugins.git.GitSCM
ActiveMQ-Artemis-Nightly-Regression-Test
		hudson.plugins.git.GitSCM
ActiveMQ-Artemis-PR-Build
		hudson.plugins.git.GitSCM
ActiveMQ-Java8
		hudson.plugins.git.GitSCM
ActiveMQ-Java8-5.15.x
		hudson.plugins.git.GitSCM
ActiveMQ-Java8-All-UnitTests
		hudson.plugins.git.GitSCM
ActiveMQ-openwire
		hudson.plugins.git.GitSCM
ActiveMQ-PR-Build
		hudson.plugins.git.GitSCM
Act

		hudson.plugins.git.GitSCM
beam_BiqQueryIO_Batch_Performance_Test_Java_Json
		hudson.plugins.git.GitSCM
beam_BiqQueryIO_Batch_Performance_Test_Java_PR
		hudson.plugins.git.GitSCM
beam_BiqQueryIO_Read_Performance_Test_Python
		hudson.plugins.git.GitSCM
beam_BiqQueryIO_Read_Performance_Test_Python_PR
		hudson.plugins.git.GitSCM
beam_BiqQueryIO_Stream_Performance_Test_Java
		hudson.plugins.git.GitSCM
beam_BiqQueryIO_Stream_Performance_Test_Java_PR
		hudson.plugins.git.GitSCM
beam_BiqQueryIO_Streaming_Performance_Test_Java
		hudson.plugins.git.GitSCM
beam_BiqQueryIO_Write_Performance_Test_Python_Batch
		hudson.plugins.git.GitSCM
beam_BiqQueryIO_Write_Performance_Test_Python_Batch_PR
		hudson.plugins.git.GitSCM
beam_CancelStaleDataflowJobs
		hudson.plugins.git.GitSCM
beam_conditional_phrase_triggering_test
		hudson.plugins.git.GitSCM
beam_Dependency_Check
		hudson.plugins.git.GitSCM
beam_Docker_Image_cleanup_beam6
		hudson.plugins.git.GitSCM
beam_Inventory_apache-beam-jenkins-1
		hudson.pl

		hudson.plugins.git.GitSCM
beam_PerformanceTests_MongoDBIO_IT
		hudson.plugins.git.GitSCM
beam_PerformanceTests_ParquetIOIT
		hudson.plugins.git.GitSCM
beam_PerformanceTests_ParquetIOIT_HDFS
		hudson.plugins.git.GitSCM
beam_PerformanceTests_Python
		hudson.plugins.git.GitSCM
beam_PerformanceTests_Python35
		hudson.plugins.git.GitSCM
beam_PerformanceTests_Spark
		hudson.plugins.git.GitSCM
beam_PerformanceTests_TextIOIT
		hudson.plugins.git.GitSCM
beam_PerformanceTests_TextIOIT_HDFS
		hudson.plugins.git.GitSCM
beam_PerformanceTests_TFRecordIOIT
		hudson.plugins.git.GitSCM
beam_PerformanceTests_WordCountIT_Py27
		hudson.plugins.git.GitSCM
beam_PerformanceTests_WordCountIT_Py35
		hudson.plugins.git.GitSCM
beam_PerformanceTests_WordCountIT_Py36
		hudson.plugins.git.GitSCM
beam_PerformanceTests_WordCountIT_Py37
		hudson.plugins.git.GitSCM
beam_PerformanceTests_XmlIOIT
		hudson.plugins.git.GitSCM
beam_PerformanceTests_XmlIOIT_HDFS
		hudson.plugins.git.GitSCM
beam_PostCommit_Go
		hudson.plugi

		hudson.plugins.git.GitSCM
beam_PostCommit_Python_VR_Spark
		hudson.plugins.git.GitSCM
beam_PostCommit_Python_VR_Spark_PR
		hudson.plugins.git.GitSCM
beam_PostCommit_SQL
		hudson.plugins.git.GitSCM
beam_PostCommit_SQL_PR
		hudson.plugins.git.GitSCM
beam_PostCommit_Website_Merge_PR
		hudson.plugins.git.GitSCM
beam_PostCommit_Website_Publish
		hudson.plugins.git.GitSCM
beam_PostCommit_Website_Publish_PR
		hudson.plugins.git.GitSCM
beam_PostCommit_Website_Test
		hudson.plugins.git.GitSCM
beam_PostCommit_Website_Test_PR
		hudson.plugins.git.GitSCM
beam_PostCommit_XVR_Flink
		hudson.plugins.git.GitSCM
beam_PostCommit_XVR_Flink_PR
		hudson.plugins.git.GitSCM
beam_PostCommit_XVR_Spark
		hudson.plugins.git.GitSCM
beam_PostCommit_XVR_Spark_PR
		hudson.plugins.git.GitSCM
beam_PostRelease_NightlySnapshot
		hudson.plugins.git.GitSCM
beam_PostRelease_Python_Candidate
		hudson.plugins.git.GitSCM
beam_PreCommit_Community Metrics_Commit
		hudson.plugins.git.GitSCM
beam_PreCommit_Community Metrics_Phr

		hudson.plugins.git.GitSCM
bookkeeper_postcommit_master_java11_test
		hudson.plugins.git.GitSCM
bookkeeper_postcommit_master_java8
		hudson.plugins.git.GitSCM
bookkeeper_postcommit_master_python
		hudson.plugins.git.GitSCM
bookkeeper_postcommit_publish_website
		hudson.plugins.git.GitSCM
bookkeeper_postcommit_validation_master
		hudson.plugins.git.GitSCM
bookkeeper_release_branch_46
		hudson.plugins.git.GitSCM
bookkeeper_release_branch_47_integrationtests
		hudson.plugins.git.GitSCM
bookkeeper_release_branch_47_java8
		hudson.plugins.git.GitSCM
bookkeeper_release_branch_47_java9
		hudson.plugins.git.GitSCM
bookkeeper_release_branch_48_integrationtests
		hudson.plugins.git.GitSCM
bookkeeper_release_branch_48_java8
		hudson.plugins.git.GitSCM
bookkeeper_release_branch_48_java9
		hudson.plugins.git.GitSCM
bookkeeper_release_nightly_snapshot
		hudson.plugins.git.GitSCM
brooklyn
brooklyn-client
brooklyn-dist
brooklyn-docker-test
		hudson.scm.NullSCM
brooklyn-docs
brooklyn-integration-tests

...

		hudson.plugins.git.GitSCM
polygene-java
Polygene-support-2.1-java7-check
		hudson.plugins.git.GitSCM
Polygene-support-2.1-java7-checkDists
		hudson.plugins.git.GitSCM
portals-applications-dbbrowser-trunk
		hudson.scm.SubversionSCM
portals-applications-demo-trunk
		hudson.scm.SubversionSCM
portals-applications-gems-trunk
		hudson.scm.SubversionSCM
portals-applications-logging-trunk
		hudson.scm.SubversionSCM
portals-applications-pom-trunk
		hudson.scm.SubversionSCM
portals-applications-rss-trunk
		hudson.scm.SubversionSCM
portals-applications-webcontent-trunk
		hudson.scm.SubversionSCM
portals-bridges-common-trunk
		hudson.scm.SubversionSCM
portals-bridges-jsf-trunk
		hudson.scm.SubversionSCM
portals-bridges-pom-trunk
		hudson.scm.SubversionSCM
portals-bridges-script-trunk
		hudson.scm.SubversionSCM
portals-bridges-velocity-trunk
		hudson.scm.SubversionSCM
portals-jetspeed-2-applications-j2-admin-2.2.3
		hudson.scm.SubversionSCM
portals-jetspeed-2-applications-j2-admin-trunk
		hudson.

		hudson.plugins.git.GitSCM
redback-components
		hudson.scm.SubversionSCM
Reef-pull-request-ubuntu
		hudson.plugins.git.GitSCM
Reef-pull-request-windows
		hudson.plugins.git.GitSCM
Reef-pull-request-windows2
		hudson.plugins.git.GitSCM
REEF-pull-request-windows3
		hudson.plugins.git.GitSCM
replay_extcdi117
		hudson.scm.NullSCM
river-2.2-qa-jdk7
		hudson.scm.SubversionSCM
river-2.2-qa-win
		hudson.scm.SubversionSCM
river-ConstraintTests
		hudson.scm.SubversionSCM
River-dev-jdk7
		hudson.scm.SubversionSCM
River-dev-jdk8
		hudson.scm.SubversionSCM
river-discoveryTests
		hudson.scm.SubversionSCM
river-EventMailboxServiceTests
		hudson.scm.SubversionSCM
river-IIOPTests
		hudson.scm.SubversionSCM
river-J9-JavaSpaceTests
		hudson.scm.SubversionSCM
river-JavaSpacesTests
		hudson.scm.SubversionSCM
river-JERITests
		hudson.scm.SubversionSCM
river-JoinManagerTests
		hudson.scm.SubversionSCM
river-JRMPactivationTests
		hudson.scm.SubversionSCM
River-ldj-jdk7
		hudson.scm.SubversionSCM
river-LeaseT

		hudson.plugins.git.GitSCM
Tamaya-Site-jBake-Master
		hudson.plugins.git.GitSCM
Tamaya-Site-Master
		hudson.plugins.git.GitSCM
tap-ci
		hudson.plugins.git.GitSCM
tapestry-4.1-trunk
		hudson.scm.SubversionSCM
tapestry-5.1-freestyle
		hudson.scm.SubversionSCM
tapestry-5.4.x-freestyle
		hudson.plugins.git.GitSCM
tapestry-trunk-freestyle
		hudson.plugins.git.GitSCM
tapestry-trunk-freestyle-java-9
		hudson.plugins.git.GitSCM
Test
		hudson.scm.NullSCM
test-by-sdp
		hudson.scm.NullSCM
test-cpus
		hudson.scm.NullSCM
test-github-branch-source-plugin
test-ulimit
		hudson.scm.NullSCM
TestBuilds
		hudson.scm.SubversionSCM
Tez-0.5
		hudson.plugins.git.GitSCM
Tez-Build
		hudson.plugins.git.GitSCM
Tez-Build-branch-0.6
		hudson.plugins.git.GitSCM
Tez-Build-branch-0.7
		hudson.plugins.git.GitSCM
Tez-Build-branch-0.8
		hudson.plugins.git.GitSCM
Tez-Build-branch-tez-2003
		hudson.plugins.git.GitSCM
Tez-Build-branch-TEZ-2980
		hudson.plugins.git.GitSCM
Tez-Build-Hadoop-2.2
		hudson.plugins.git.GitSCM
Tez

		hudson.plugins.git.GitSCM
ZooKeeper-trunk-java13
		hudson.plugins.git.GitSCM
ZooKeeper-trunk-java9
		hudson.plugins.git.GitSCM
ZooKeeper-trunk-jdk8
		hudson.plugins.git.GitSCM
ZooKeeper-trunk-openjdk7
		hudson.plugins.git.GitSCM
ZooKeeper-trunk-owasp
		hudson.plugins.git.GitSCM
Zookeeper-trunk-single-thread
		hudson.plugins.git.GitSCM
ZooKeeper-trunk-solaris
		hudson.plugins.git.GitSCM
ZooKeeper-trunk-windows-cmake
		hudson.plugins.git.GitSCM
ZooKeeper_branch33
		hudson.plugins.git.GitSCM
ZooKeeper_branch33_solaris
		hudson.plugins.git.GitSCM
ZooKeeper_branch34
		hudson.plugins.git.GitSCM
ZooKeeper_branch34_java10
		hudson.plugins.git.GitSCM
ZooKeeper_branch34_java11
		hudson.plugins.git.GitSCM
ZooKeeper_branch34_java9
		hudson.plugins.git.GitSCM
ZooKeeper_branch34_jdk7
		hudson.plugins.git.GitSCM
ZooKeeper_branch34_jdk8
		hudson.plugins.git.GitSCM
ZooKeeper_branch34_openjdk12
		hudson.plugins.git.GitSCM
ZooKeeper_branch34_openjdk7
		hudson.plugins.git.GitSCM
ZooKeeper_branch34_openj

JenkinsException: job[jclouds-2.2.x] does not exist

All jobs and their respective SCM class. There are 4 main classes: GitSCM, SubversionSCM, NullSCM and MultiSCM.

In [7]:
server.get_job_info('aurora-packaging-nightly', depth=3)['scm']

{'_class': 'org.jenkinsci.plugins.multiplescms.MultiSCM',
 'browser': None,
 'type': 'org.jenkinsci.plugins.multiplescms.MultiSCM',
 'configuredSCMs': [{'_class': 'hudson.plugins.git.GitSCM',
   'browser': {'_class': 'hudson.plugins.git.browser.GithubWeb'},
   'type': 'hudson.plugins.git.GitSCM',
   'branches': [{'name': '*/master'}],
   'mergeOptions': {'fastForwardMode': '--ff',
    'mergeStrategy': 'default',
    'mergeTarget': None,
    'remoteBranchName': None},
   'userRemoteConfigs': [{'credentialsId': None,
     'name': None,
     'refspec': None,
     'url': 'https://github.com/apache/aurora.git'}]},
  {'_class': 'hudson.plugins.git.GitSCM',
   'browser': {'_class': 'hudson.plugins.git.browser.GithubWeb'},
   'type': 'hudson.plugins.git.GitSCM',
   'branches': [{'name': '*/master'}],
   'mergeOptions': {'fastForwardMode': '--ff',
    'mergeStrategy': 'default',
    'mergeTarget': None,
    'remoteBranchName': None},
   'userRemoteConfigs': [{'credentialsId': None,
     'name':

Example of MultiSCM class.

In [16]:
changeSet = server.get_job_info('commons-bcel', depth=2)['builds'][1]['changeSet']

In [19]:
print(type(changeSet['items']))
print(len(changeSet['items']))
changeSet['items'][1]


<class 'list'>
2


{'_class': 'hudson.plugins.git.GitChangeSet',
 'affectedPaths': ['src/main/java/org/apache/bcel/classfile/Utility.java',
  'src/site/resources/bcel5-bcel6-clirr-report.html',
  'src/main/java/org/apache/bcel/package.html',
  'src/main/java/org/apache/bcel/Constants.java',
  'src/main/java/org/apache/bcel/classfile/BootstrapMethod.java',
  'src/main/java/org/apache/bcel/generic/INVOKEDYNAMIC.java',
  'src/main/java/org/apache/bcel/generic/INVOKEINTERFACE.java',
  'src/site/xdoc/faq.xml',
  'src/main/java/org/apache/bcel/classfile/MethodParameters.java',
  'src/main/java/org/apache/bcel/generic/INVOKESTATIC.java',
  'src/main/java/org/apache/bcel/generic/INVOKESPECIAL.java',
  'src/main/java/org/apache/bcel/classfile/ConstantInvokeDynamic.java',
  'src/main/java/org/apache/bcel/classfile/MethodParameter.java',
  'src/main/java/org/apache/bcel/classfile/BootstrapMethods.java',
  'src/main/java/org/apache/bcel/Const.java',
  'src/main/java/org/apache/bcel/classfile/JavaClass.java',
  'src/

There can be multiple commits related to a specific build (usually one).

In [20]:
server.get_job_info('commons-bcel', depth=2)['builds'][9]['changeSet'] # where stores the commit infos

{'_class': 'hudson.plugins.git.GitChangeSetList',
 'items': [{'_class': 'hudson.plugins.git.GitChangeSet',
   'affectedPaths': ['src/examples/HelloWorldBuilder.java'],
   'commitId': '0d65a2718b4171a947388f20b59b4e1ae51d4a74',
   'timestamp': 1575905180000,
   'author': {'absoluteUrl': 'https://builds.apache.org/user/garydgregory',
    'fullName': 'garydgregory'},
   'authorEmail': 'garydgregory@users.noreply.github.com',
   'comment': 'Replaced deprecated constants in examples (#38)\n',
   'date': '2019-12-09 10:26:20 -0500',
   'id': '0d65a2718b4171a947388f20b59b4e1ae51d4a74',
   'msg': 'Replaced deprecated constants in examples (#38)',
   'paths': [{'editType': 'edit',
     'file': 'src/examples/HelloWorldBuilder.java'}]},
  {'_class': 'hudson.plugins.git.GitChangeSet',
   'affectedPaths': ['src/changes/changes.xml',
    'src/examples/HelloWorldBuilder.java'],
   'commitId': '70f4133d789b2b64eb440361683f08dd6ee6a7a3',
   'timestamp': 1575905784000,
   'author': {'absoluteUrl': 'http

In [54]:
len(server.get_job_info_regex('Apache Commons VFS', depth = 2, folder_depth=0)[0]['builds'])

100

In [47]:
len(server.get_job_info('Cassandra-2.2-dtest-novnode', depth = 2, fetch_all_builds=True)['builds'])

26

### Check total number of builds of each job

In [53]:
for i in range(len(jobs_2)):
    name = jobs_2[i]['name']
    print(name, "\t\t #: ", end = "")
    job_info = server.get_job_info(name, depth = 2, fetch_all_builds=True)
    if 'builds' in job_info:
        l = len(job_info['builds'])
        print(l)
    else:
        print("")

2.2.x 		 #: 
Abdera-trunk 		 #: 1
Accumulo-1.9 		 #: 5
Accumulo-2.0 		 #: 5
Accumulo-Master 		 #: 5
Accumulo-master-IT 		 #: 5
Accumulo-Maven-Plugin-Master 		 #: 6
Accumulo-Maven-Plugin-Pull-Requests 		 #: 1
Accumulo-PR-Builder 		 #: 1
Accumulo-Pull-Requests 		 #: 15
ACE-trunk 		 #: 5
ActiveMQ Protocol Buffer 		 #: 5
ActiveMQ-Artemis-Deploy 		 #: 6
ActiveMQ-Artemis-Master 		 #: 6
ActiveMQ-Artemis-Nightly-Regression-Test 		 #: 6
ActiveMQ-Artemis-PR-Build 		 #: 10
ActiveMQ-Java8 		 #: 7
ActiveMQ-Java8-5.15.x 		 #: 7
ActiveMQ-Java8-All-UnitTests 		 #: 7
ActiveMQ-openwire 		 #: 0
ActiveMQ-PR-Build 		 #: 5
ActiveMQ-SysTest-Trunk 		 #: 1
ActiveMQ-Trunk-Deploy 		 #: 2
airavata 		 #: 0
airavata-dev 		 #: 0
airavata-master 		 #: 0
airavata-push 		 #: 0
Allura 		 #: 5
Allura-licensing-check 		 #: 10
AMBARI-22504-test-patch 		 #: 6
Ambari-branch-1.7.0 		 #: 1
Ambari-branch-1.7.0-docker 		 #: 1
Ambari-branch-2.0.0 		 #: 1
Ambari-branch-2.1 		 #: 1
Ambari-branch-2.2 		 #: 1
Ambari-branch-2.5 		 #: 

KeyboardInterrupt: 

Most have < 50 builds, for the first dozens of jobs, there is only Apache Commons VFS with 108 builds.